
## Brian's Week 3 Homework for Coursera Capstone part 3 ##

start off by doing everything from Brian's hw3 part2, but don't print as much out

scrape the Wikipedia page

In [1]:
#scrape table on wikipedia page
import requests
import lxml.html as lh
import pandas as pd

url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(url)
doc = lh.fromstring(page.content)
tr_elements = doc.xpath('//tr')
[len(T) for T in tr_elements[:12]]

tr_elements = doc.xpath('//tr')
#Create empty list
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    #print (i,name)
    col.append((name,[]))

In [2]:
#len(tr_elements)

In [3]:

#print('parsing start')
#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
   # print('parsing', j)
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 10, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        data
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1
#print('done parsing table')        

In [4]:
[len(C) for (title,C) in col]

[288, 288, 288]

In [5]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

In [6]:
#df.head()

Looks like we have the columns in the wrong order, so let's fix that now

In [7]:
#looks like we have the columns in the wrong order, so fix that

df = df[['Postcode', 'Borough', 'Neighbourhood\n']]#notice the \n in the third column - that took me a while to figure out
#df.head()

rename the columns to match the correct project name requirements

In [8]:
#rename columns to the project requirements
df.rename({'Neighbourhood\n': 'Neighborhood'}, axis='columns', inplace=True)
df.rename({'Postcode': 'PostalCode'}, axis='columns', inplace=True)
#df.head()

Clean up the dataframe

In [9]:
#clean up df and keep the old one
df_unclean=df
#df.shape
#take out the newlines in the data
df = df.replace('\n','', regex=True)
#lets look at the values we have and see that they make sense
#print(df['Neighborhood'].unique())
#print(df['Borough'].unique())
#print(df['PostalCode'].unique())

Get rid of rowns where Borough is not assigned

In [10]:
# get rid of rows where the Borough is not assigned
print(df.shape)
df=df[df['Borough']!='Not assigned']
#print(df.shape)
#print(df['Borough'].unique())

(288, 3)


If a neighborhood is not assigned, change it to match the borough name

In [11]:
#If a neighborhood is not assigned, change it to be the same as the borough
df.Neighborhood.value_counts()['Not assigned'] #how many do we have?
df.loc[df.Neighborhood == 'Not assigned', 'Neighborhood'] = df['Borough']
#df[0:10] #you can see that the Queen's Park neighborhood was assigned

In [12]:
all_postalcodes = df.PostalCode.unique()
#len(all_postalcodes)

combine multiple rows for a postalcode - combine the neighborhoods into commas separated lists

In [13]:
for postalcode in all_postalcodes:
    #print('\n\npostalcode=', postalcode, '\n')
    neighborhoods_to_combine = df[df['PostalCode']==postalcode].Neighborhood.unique()
    number_of_neighborhoods_to_combine = neighborhoods_to_combine.size
    neighborhoods_to_combine_csv_str = ', '.join(map(str, neighborhoods_to_combine))
                                      
    #print(number_of_neighborhoods_to_combine, ' - ', neighborhoods_to_combine_csv_str)
    #rows = df[df['PostalCode']==postalcode]
    #rows.Neighborhood=neighborhoods_to_combine_csv_str
    df.loc[df.PostalCode == postalcode, 'Neighborhood'] = neighborhoods_to_combine_csv_str
    
    #print(rows)
    #new_df.append(rows)


Combined the neighborhoods by postalcode, but didn't get rid of duplicate rows generated yet - let's get rid of duplicates now

In [14]:
print(df.shape)
df.drop_duplicates(inplace=True)
print(df.shape)
#df

(211, 3)
(103, 3)


fix the index

In [15]:
df.reset_index(inplace=True, drop=True)
#df

Last step for part 1 - print the datafram shape

In [16]:
df.shape

(103, 3)

done with Brian's hw3 part 1

## start the new stuff for Brian's hw3 part2 ##


Add latitude and longitude

In [17]:
#add column headers for latitude and longitude
header_list = ['PostalCode','Borough','Neighborhood','Latitude','Longitude']
df = df.reindex(columns = header_list)

The geocoder package never gave me results - it is too unreliable so I am commenting it

In [18]:
#I implemented the geocoder latitude and longitude,but it never returned results
'''
!conda install -c conda-forge geocoder -y
import geocoder # import geocoder

for postal_code in all_postalcodes:
    # initialize your variable to None
    lat_lng_coords = None
    print('\n',postal_code)
    # loop until you get the coordinates
    count=0
    while(lat_lng_coords is None):
      g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
      lat_lng_coords = g.latlng
      print('.',count,'.',postal_code,'.',g.latlng,'.')
      count=count+1
        
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    print('\n',latitude,' ', longitude,'\n')
    df.loc[df.PostalCode == postalcode, 'Latitude','Longitude'] = latitude,longitude
'''

"\n!conda install -c conda-forge geocoder -y\nimport geocoder # import geocoder\n\nfor postal_code in all_postalcodes:\n    # initialize your variable to None\n    lat_lng_coords = None\n    print('\n',postal_code)\n    # loop until you get the coordinates\n    count=0\n    while(lat_lng_coords is None):\n      g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))\n      lat_lng_coords = g.latlng\n      print('.',count,'.',postal_code,'.',g.latlng,'.')\n      count=count+1\n        \n    latitude = lat_lng_coords[0]\n    longitude = lat_lng_coords[1]\n    print('\n',latitude,' ', longitude,'\n')\n    df.loc[df.PostalCode == postalcode, 'Latitude','Longitude'] = latitude,longitude\n"

Get the latitudes and longitudes from the csv file since geocoder didn't work

In [19]:
lat_long_data = pd.read_csv('https://cocl.us/Geospatial_data')
lat_long_data.head()
for i in range(lat_long_data.Latitude.count()):
    lat = lat_long_data.iloc[i]['Latitude']
    lon = lat_long_data.iloc[i]['Longitude']
    pc = lat_long_data.iloc[i]['Postal Code']
    #print('\n',pc,'-',lat,'-',lon,'\n')
    df.loc[df.PostalCode == pc, 'Latitude'] = lat
    df.loc[df.PostalCode == pc, 'Longitude'] = lon

In [20]:
#does it look right?
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


show the dataframe with the latitude and the longitude

In [21]:
#df

## start Brian's hw3 part3 ##

In [22]:
#imports
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

Solving environment: done

# All requested packages already installed.



I googled the initial latitude and longitude of Toronto

In [23]:
print('Libraries imported.')

# create map of Toronto using latitude and longitude values
toronto_lat = 43.6532
toronto_lon = -79.3832
latitude = toronto_lat#messy renaming butit works
longitude = toronto_lon#messy renaming but it works
map_toronto = folium.Map(location=[toronto_lat, toronto_lon], zoom_start=11)

neighborhoods = df
# add markers to map

for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Libraries imported.


Let's reduce our data to borough's containing the word "Toronto"

In [24]:
neighborhoods = neighborhoods[neighborhoods.Borough.str.contains("Toronto")]
neighborhoods.head()#check if it looks right

,PostalCode,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


fix the index

In [25]:
neighborhoods.reset_index(inplace=True,drop=True)
neighborhoods.head()#check if it looks right

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
1,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


redo map

In [26]:
new_map_toronto = folium.Map(location=[toronto_lat, toronto_lon], zoom_start=11)

for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(new_map_toronto)  
    
new_map_toronto

Define fourquare credentials

In [27]:
# The code was removed by Watson Studio for sharing.

Let's explore the first neighborhood in our dataframe.

In [28]:
toronto_data = neighborhoods
toronto_data.loc[0, 'Neighborhood']

'Harbourfront, Regent Park'

Get the neighborhood's latitude and longitude values.

In [29]:
neighborhood_latitude = toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Harbourfront, Regent Park are 43.6542599, -79.3606359.


In [30]:
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=0C4GFTHUCEID10D2EF5OUUVTPDE3LRKIKOCWYOXSVTN0L3FB&client_secret=UC52VVB1WLAD40QJIBUYHX54AMWFXPGJEMM4C0OZRGCOHWFZ&v=20180605&ll=43.6542599,-79.3606359&radius=500&limit=15'

In [31]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [32]:

toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Adelaide, King, Richmond
Dovercourt Village, Dufferin
Harbourfront East, Toronto Islands, Union Station
Little Portugal, Trinity
The Danforth West, Riverdale
Design Exchange, Toronto Dominion Centre
Brockton, Exhibition Place, Parkdale Village
The Beaches West, India Bazaar
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North, Forest Hill West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
Harbord, University of Toronto
Runnymede, Swansea
Moore Park, Summerhill East
Chinatown, Grange Park, Kensington Market
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown, St. James Town
Fir

check the size ofthe resulting dataframe

In [33]:
print(toronto_venues.shape)
toronto_venues.head()

(492, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Harbourfront, Regent Park",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Harbourfront, Regent Park",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Harbourfront, Regent Park",43.65426,-79.360636,Toronto Cooper Koo Family Cherry St YMCA Centre,43.653191,-79.357947,Gym / Fitness Center
3,"Harbourfront, Regent Park",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Harbourfront, Regent Park",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot


howmanhy venuresreturned foreach neighborhood

In [34]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",15,15,15,15,15,15
Berczy Park,15,15,15,15,15,15
"Brockton, Exhibition Place, Parkdale Village",15,15,15,15,15,15
Business Reply Mail Processing Centre 969 Eastern,15,15,15,15,15,15
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",14,14,14,14,14,14
"Cabbagetown, St. James Town",15,15,15,15,15,15
Central Bay Street,15,15,15,15,15,15
"Chinatown, Grange Park, Kensington Market",15,15,15,15,15,15
Christie,15,15,15,15,15,15


In [35]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 148 uniques categories.


Analyze eachneighborhood

In [36]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,...,Taco Place,Taiwanese Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Let's see the new dataframe size

In [37]:
toronto_onehot.shape

(492, 148)

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [38]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,...,Taco Place,Taiwanese Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar
0,"Adelaide, King, Richmond",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.066667,0.000000,0.000000
1,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.066667,0.066667,0.000000,0.000000,0.00,0.066667,0.000000,0.000000
2,"Brockton, Exhibition Place, Parkdale Village",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
3,Business Reply Mail Processing Centre 969 Eastern,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.000000,0.071429,0.071429,0.142857,0.214286,0.142857,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
5,"Cabbagetown, St. James Town",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
6,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
7,"Chinatown, Grange Park, Kensington Market",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.133333,0.000000
8,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
9,Church and Wellesley,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.066667,0.000000,0.000000,0.066667,0.00,0.000000,0.000000,0.000000


Let's confirm the new size

In [39]:
toronto_grouped.shape

(38, 148)

Let's print each neighborhood along with the top 5 most common venues

In [40]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
              venue  freq
0        Steakhouse  0.13
1      Concert Hall  0.07
2  Asian Restaurant  0.07
3       Opera House  0.07
4         Speakeasy  0.07


----Berczy Park----
            venue  freq
0  Farmers Market  0.13
1            Park  0.07
2          Museum  0.07
3        Beer Bar  0.07
4    Liquor Store  0.07


----Brockton, Exhibition Place, Parkdale Village----
                venue  freq
0         Coffee Shop  0.13
1      Breakfast Spot  0.13
2                 Gym  0.07
3  Italian Restaurant  0.07
4        Climbing Gym  0.07


----Business Reply Mail Processing Centre 969 Eastern----
           venue  freq
0    Yoga Studio  0.07
1  Auto Workshop  0.07
2    Pizza Place  0.07
3     Restaurant  0.07
4  Burrito Place  0.07


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
              venue  freq
0   Airport Service  0.21
1    Airport Lounge  0.14
2  Airport Terminal  0.14
3  

### Let's put that into a pandas dataframe ###

First, let's write a function to sort the venues in descending order.

In [41]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [42]:
import numpy as np

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Steakhouse,Opera House,Asian Restaurant,Café,Coffee Shop,Pizza Place,Concert Hall,Plaza,Seafood Restaurant,Speakeasy
1,Berczy Park,Farmers Market,Beer Bar,Concert Hall,Steakhouse,Museum,Liquor Store,Park,Cocktail Bar,Tea Room,Thai Restaurant
2,"Brockton, Exhibition Place, Parkdale Village",Coffee Shop,Breakfast Spot,Bar,Stadium,Furniture / Home Store,Caribbean Restaurant,Performing Arts Venue,Pet Store,Café,Restaurant
3,Business Reply Mail Processing Centre 969 Eastern,Yoga Studio,Auto Workshop,Garden Center,Fast Food Restaurant,Farmers Market,Comic Shop,Light Rail Station,Park,Pizza Place,Burrito Place
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Service,Airport Lounge,Airport Terminal,Plane,Airport,Airport Food Court,Sculpture Garden,Boat or Ferry,Boutique,Harbor / Marina


### Cluster Neighborhoods ###

Run k-means to cluster the neighborhood into 5 clusters.

In [43]:
from sklearn.cluster import KMeans# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 1, 1, 1, 1, 3, 3, 3, 3, 1], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [44]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636,3,Coffee Shop,Breakfast Spot,Chocolate Shop,Farmers Market,Park,Spa,Bakery,Restaurant,Gym / Fitness Center,Pub
1,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,3,Café,Burrito Place,Comic Shop,Music Venue,Clothing Store,Theater,Thai Restaurant,Tea Room,Taco Place,Ramen Restaurant
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,3,Gastropub,Coffee Shop,Restaurant,Japanese Restaurant,Church,BBQ Joint,Middle Eastern Restaurant,Cosmetics Shop,Creperie,Italian Restaurant
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,3,Pub,Astrologer,Health Food Store,Convenience Store,Dessert Shop,Deli / Bodega,Dance Studio,Cuban Restaurant,Creperie,Cosmetics Shop
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,1,Farmers Market,Beer Bar,Concert Hall,Steakhouse,Museum,Liquor Store,Park,Cocktail Bar,Tea Room,Thai Restaurant


Finally, let's visualize the resulting clusters

In [45]:
# create map
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters